In [166]:
import requests
import json
import pandas as pd

# HDB Resale Flat Price Data Collection
This notebook pulls resale flat data from the Singapore government's public API, cleans it, and combines multiple datasets into one.

In [167]:
collection_id = 189          
url = "https://api-production.data.gov.sg/v2/public/api/collections/{}/metadata".format(collection_id)
        
response = requests.get(url)

data = response.json()
#print(json.dumps(data, indent=2))

for id in data['data']['collectionMetadata']['childDatasets']:
    url = f"https://api-production.data.gov.sg/v2/public/api/datasets/{id}/metadata"
    
    response = requests.get(url)
    print(f"{id} : {response.json()['data']['name']}")

d_8b84c4ee58e3cfc0ece0d773c8ca6abc : Resale flat prices based on registration date from Jan-2017 onwards
d_43f493c6c50d54243cc1eab0df142d6a : Resale Flat Prices (Based on Approval Date), 2000 - Feb 2012
d_2d5ff9ea31397b66239f245f57751537 : Resale Flat Prices (Based on Registration Date), From Mar 2012 to Dec 2014
d_ebc5ab87086db484f88045b47411ebc5 : Resale Flat Prices (Based on Approval Date), 1990 - 1999
d_ea9ed51da2787afaf8e51f827c304208 : Resale Flat Prices (Based on Registration Date), From Jan 2015 to Dec 2016


In [ ]:
def fetch_and_store_records(id, limit = 10000):   
    offset = 0
    all_records = []

    while True:
        url = f"https://data.gov.sg/api/action/datastore_search?resource_id={id}&limit={limit}&offset={offset}"
        response = requests.get(url)
        records = response.json()['result']['records']
        
        all_records.extend(records)
        
        if len(records) < limit:
            break
        
        offset += limit 

    return pd.DataFrame(all_records)

In [173]:
dataset_ids = [
    "d_ea9ed51da2787afaf8e51f827c304208",  # 2015 to 2016
    "d_8b84c4ee58e3cfc0ece0d773c8ca6abc" # 2017 onwards
]

df1, df2 = fetch_and_store_records(dataset_ids[0]), fetch_and_store_records(dataset_ids[1])
df2['remaining_lease'] = df2['remaining_lease'].str.extract(r"(\d+)\s+years")
hdb_resale_data = pd.concat([df1, df2], axis=0, ignore_index=True)

In [174]:
hdb_resale_data.head()

,_id,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,1,2015-01,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,07 TO 09,60,Improved,1986,70,255000
1,2,2015-01,ANG MO KIO,3 ROOM,541,ANG MO KIO AVE 10,01 TO 03,68,New Generation,1981,65,275000
2,3,2015-01,ANG MO KIO,3 ROOM,163,ANG MO KIO AVE 4,01 TO 03,69,New Generation,1980,64,285000
3,4,2015-01,ANG MO KIO,3 ROOM,446,ANG MO KIO AVE 10,01 TO 03,68,New Generation,1979,63,290000
4,5,2015-01,ANG MO KIO,3 ROOM,557,ANG MO KIO AVE 10,07 TO 09,68,New Generation,1980,64,290000
